In [ ]:
%pip install opencv-python

In [ ]:
%pip install deepface

In [ ]:
%pip install tf-keras

In [78]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [111]:
import threading
import cv2
from deepface import DeepFace
import json
import pandas as pd

In [114]:
verstappen1="./image_database/Max Verstappen/max_verstappen1.jpg"
verstappen2="./image_database/Max Verstappen/max_verstappen2.jpg"
vdraganic="./image_database/Vedran Draganic/vedran_draganic.png"

models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
  "GhostFaceNet"
]

result = DeepFace.verify(
  verstappen1,
  vdraganic,
  model_name=models[0]
)
print(json.dumps(result, indent=2))

result = DeepFace.verify(
  verstappen1,
  verstappen2,
  model_name=models[0]
)
print(json.dumps(result, indent=2))

{
  "verified": false,
  "distance": 0.9077790957691607,
  "threshold": 0.68,
  "model": "VGG-Face",
  "detector_backend": "opencv",
  "similarity_metric": "cosine",
  "facial_areas": {
    "img1": {
      "x": 251,
      "y": 95,
      "w": 274,
      "h": 274,
      "left_eye": [
        436,
        200
      ],
      "right_eye": [
        330,
        200
      ]
    },
    "img2": {
      "x": 60,
      "y": 48,
      "w": 139,
      "h": 139,
      "left_eye": [
        151,
        101
      ],
      "right_eye": [
        103,
        102
      ]
    }
  },
  "time": 0.62
}
{
  "verified": true,
  "distance": 0.46264304359120956,
  "threshold": 0.68,
  "model": "VGG-Face",
  "detector_backend": "opencv",
  "similarity_metric": "cosine",
  "facial_areas": {
    "img1": {
      "x": 251,
      "y": 95,
      "w": 274,
      "h": 274,
      "left_eye": [
        436,
        200
      ],
      "right_eye": [
        330,
        200
      ]
    },
    "img2": {
      "x": 630,
  

In [107]:
face_match_name = "Unknown"

# Verify face
def check_face(frame):
    global face_match_name
    try:
        # Perform face search in the image database
        results = DeepFace.find(frame, db_path="./image_database")

        # Handle the case where `results` is a DataFrame
        if isinstance(results, list) and len(results) > 0:  # Handle list of DataFrames
            results = results[0]

        if isinstance(results, pd.DataFrame) and not results.empty:  # Ensure it's a DataFrame and not empty
            # Access the first match and extract the identity
            face_match_name = results.iloc[0]['identity'].split("\\")[1]  # Extract name from path
        else:
            face_match_name = "Unknown"
    except ValueError:
        face_match_name = "Unknown"
    except Exception as e:
        print(f"Error in check_face: {e}")
        face_match_name = "Unknown"


In [116]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

counter = 0

# Load pre-trained face detector model (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()

    if ret:
        # Run face matching every 60 frames (approximately every 2 seconds at 30 FPS)
        if counter % 60 == 0:
            try:
                threading.Thread(target=check_face, args=(frame.copy(),)).start()
            except Exception as e:
                print(f"Error starting thread: {e}")
                pass
        counter += 1

        # Convert frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        # Draw rectangles around detected faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Display the matched face name
        cv2.putText(frame, face_match_name, (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

        # Display the resulting frame
        cv2.imshow("video", frame)

    # Quit on 'q' key press
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

25-01-20 00:01:35 - Searching [[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [248 250 250]
  [247 249 249]
  [244 246 246]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [247 249 249]
  [247 249 249]
  [243 245 245]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [247 249 249]
  [245 247 247]
  [241 243 243]]] in 9 length datastore
25-01-20 00:01:38 - Searching [[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  1   0   2]
  [  0   0   0]
  [  4   0   5]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  1   0   2]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  1   0   2]
  [ 13   0